# MNIST FORWARD PASS
# Imports

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from forwardPass import forwardPass

## Test Functions

In [ ]:
# Custom test model function
# Only implements forward pass of neural network - no training done.
def test_custom(model, device, test_loader):
    test_loss = 0
    correct = 0
    i = 1
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            batch = data.size()[0]

            # Custom forward pass function
            output = model.forward_pass(data, batch)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            print(f"ITERATION {i}: Accuracy = {correct/i:.2f}% (cumulative = {correct})")
            i = i+1
    test_loss /= len(test_loader.dataset)

    print(f"Loss: {test_loss}   Accuracy: {100 * correct / len(test_loader.dataset)}%\n")


# Test original/default model forward pass.
def test(model, device, test_loader):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model.model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print(f"Loss: {test_loss}   Accuracy: {100 * correct / len(test_loader.dataset)}%\n")

## Loading Model

In [ ]:
torch.set_default_dtype(torch.float32)
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST dataset with train/test split sets.
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=trans),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=trans),
    batch_size=100, shuffle=True)

idim = next(iter(train_loader))[0][0].size()[1]
ifmap = next(iter(train_loader))[0][0].size()[0]
fc2_nodes = len(torch.unique(train_loader.dataset.targets))

## Hardware Parameters

In [ ]:
# Path for quantized and trained model
model_path = "models/mnist_model_8bit.pth"

# Choose from: 'array', 'ppq_array', 'mismatched_array'
array_choice = 'mismatched_array'

# Choose from: 'linear', 'from_table', 'gain_nl', 'noise' (Ignored if using 'mismatched array')
mac_choice = 'gain_nl'

# MAC_CONSTANTS - What to set it to depends on above choices:
#
# array_choice = 'mismatched_array' - Set to percentage mismatch (2)
# 
# mac_choice = 'from_table'         - Set to .xlsx file path containing MAC results ('../HardwareSpec/8x8_Mac_result_final.xlsx')
# mac_choice = 'gain_nl'            - Set to the gain/non-linearity constants in an array: [a1, a3] ([1.1, 1*10**-5])
# mac_choice = 'noise'              - Set to noise sigma value (1)
mac_constants = 2

## Testing

In [ ]:
# Initialize hardware model
non_linear_model = forwardPass(device, idim, ifmap, array_choice, mac_choice, mac_constants, model_path)

# Evaluate test accuracy with imported quantized model from mnist_model.pth.
test(non_linear_model, device, test_loader)

# Evaluate test accuracy of qmodel, with custom forward pass.
test_custom(non_linear_model, device, test_loader)